# Fluidum Tutorial

Fluidum is a code to solve relativistic viscous fluid-dynamic equations.

http://arxiv.org/abs/1811.01870

We will solve equations in 1+1 dimensions (r, tau) to compute transverse momentum distributions of identified particles.

In [ ]:
using Fluidum

## Fluid Properties

Here we specify the properties of the QGP: shear and bulk viscosity, and the equation of state.

In [ ]:
#define equation of state and transport coefficients
eos = FluiduMEoS()

#define transport coefficients
viscosity=QGPViscosity(0.2,0.2) #first number is eta/s
bulk=SimpleBulkViscosity(0.1,15.0) #first number is zeta/s
diffusion=ZeroDiffusion() #do not modify

#collect eos and transport coefficient in the FluidProperties struct
params=Fluidum.FluidProperties(eos,viscosity,bulk,diffusion)

## Grid and fields specification

In [ ]:
#define a radial grid
gridpoints=100
rmax=20
disc=CartesianDiscretization(OriginInterval(gridpoints,rmax)) 
#define the name and number of fields, with appropriate parity
oned_visc_hydro = oned_visc_hydro=Fields(
    NDField((:even,),(:ghost,),:temperature),
    NDField((:odd,),(:ghost,),:ur),
    NDField((:even,),(:ghost,),:piphiphi),
    NDField((:even,),(:ghost,),:pietaeta),
    NDField((:even,),(:ghost,),:piB)
    )
    
disc_fields = DiscreteFileds(oned_visc_hydro,disc,Float64) 

## Initial conditions

Generate the initial conditions profiles using the Glauber model. Generate entropy density profile for e.g. the 0-5%, 5-10%, 10-20% and 20-30% centrality classes of PbPb collisions at 5 TeV at the LHC.

In [ ]:
using MonteCarloGlauber

1) The independent variable in the code is the temperature, not the entropy. Use the equation of state of an ideal gas of non-interacting quarks (Nf=3, with u,d,s) and gluons to get the temperature profile.


In [ ]:
n1= Lead()
n2= Lead()
#You can also try other nuclei: Gold, Copper, Uranium, Xenon

#nuclear parameters (do not modify)
w= 0.5
k=1
p=1.

norm = 10 #this modifies the normalization of the entropy profile. It's a free parameter in our model.
s_NN=5000 #center-of-mass energy
fmGeV=5.
#equation of state: entropy to temperature as function of temperature


energy2(T)= #substitute entropy of the ideal gas of quarks (u,d,s) and gluons 



entropyToEnergy(T)=InverseFunction(energy2)(T)
#signature: eos,(eos),norm,n1,n2,w,k,p,sNN,[centrality classes];#events, radial grid
#notice that it's expensive to generate too many events and to make the grid too large. What's the size of your nucleus?
#this will give you a rough estimate of how much your radial grid should be large.
bg=MonteCarloGlauber.generate_bg(entropyToEnergy,norm,Lead(),Lead(),w,k,p,s_NN,[5];minBiasEvents=1000,r_grid=0:0.1:10)
#bg,twpt=MonteCarloGlauber.generate_bg_two_pt_fct(energy2,energy2,norm,n1,n2,w,k,p,s_NN,[10,20,30,40],[2];minBiasEvents=1000,r_grid=0:0.1:10)

2) Plot the temperature or the entropy in the different centrality classes. What's the difference? You can also compute the total entropy (integral over radial profile) as a function of centrality.

In [ ]:
using Plots
plot(0:0.1:10,bg[1],label="0-5%",xlabel = "r [fm/c]", ylabel="T [GeV]")

You can also look at the entropy density for a single event. This will fluctuate for each event, and doesn't have radial symmetry. This is relevant for flow coefficient calculations, which we won't consider today.

In [ ]:
#Plot 10 random events
participants=Participants(n1,n2,w,s_NN,k,p) #you can add one argument at the end to specify a fixed impact parameter
event=rand(participants,10) #second argument is the #events
profile=map(event)   do x 
    map(Iterators.product(-10:0.5:10,-10:0.5:10)) do y
        x(y...)
    end
end

In [ ]:
heatmap(-10:0.5:10,-10:0.5:10,profile[10],aspectratio=1,xrange=(-10,10),yrange=(-10,10))

We will need an interpolation of the temperature profile over a larger grid for our simulation. It is best to attach an exponential tail and an atmosphere (small offset) for numerical stability.

In [ ]:
using Interpolations

temp_fun=Interpolations.linear_interpolation(0:0.1:10,bg[1])
temp_exp_tail(x) = Fluidum.exponential_tail_pointlike.(Ref(temp_fun), x; xmax = 8)
#set the temperature field
phi=set_array((x)->temp_exp_tail(x)+0.01,:temperature,disc_fields); 
plot(x->temp_exp_tail(x)+0.01,xrange=(0,20),label="exp tail")
plot!(x->temp_exp_tail(x)+0.01,xrange=(0,20), label="exp tail + atm")
#plot!(x->temp_fun(x),xrange=(0,10), label="only trento")

## Fluid evolution

We specify a time range in which we evolve the fluid fields.

In [ ]:
#define a time range for the evolution
tspan = (0.4,5)

#evolve until tspan[2]
nofo=oneshoot(disc_fields,Fluidum.matrxi1d_visc!,params,phi,tspan)

3) Plot the fluid fields. How does the fluid velocity (2nd component of the field) change with increasing shear viscosity? What happens to the dissipative fields (components 3, 4 and 5)?

In [ ]:
#example: temperature evolution plot
radius = zeros(102)
[radius[i]= disc.grid[i][1] for i in eachindex(disc.grid[1:end])] 
begin
    plot(radius[2:end-1],nofo(tspan[1])[1,2:end-1], label="t = "*string(tspan[1])*" fm/c")
    plot!(radius[2:end-1],nofo(1)[1,2:end-1])
    plot!(radius[2:end-1],nofo(2)[1,2:end-1])
    plot!(radius[2:end-1],nofo(tspan[2])[1,2:end-1], xlabel="r [fm/c]",ylabel = "T [GeV]")
end

## Freeze out

We are now going to compute observables at a freeze out surface, specified by a freeze out temperature (you can use T=0.140 GeV and T=0.156 GeV).


In [ ]:
#specify freeze-out temperature with the optional argument Tfo (in GeV)
tspan = (0.4,30)

fo =freeze_out_routine(disc_fields,Fluidum.matrxi1d_visc!,params,phi,tspan;Tfo=0.140)
fo2 =freeze_out_routine(disc_fields,Fluidum.matrxi1d_visc!,params,phi,tspan;Tfo=0.156)

In [ ]:
Fluidum.save_kernels(;Tmin=1400,Tmax=1560,dT=160, type = "thermal", file_path=pwd()*"/kernels/", MC = false)    
Fluidum.save_kernels(;Tmin=1400,Tmax=1560,dT=160, type = "total", file_path=pwd()*"/kernels/", MC = false)    


4) Plot the particle spectra for pions, kaons and protons. Which species is more abundant? Compute the multiplicity of a particle species. 
5) How does changing the freeze-out temperature affect the spectra? 
6) Plot the multiplicities as a function of centrality. How does this relate to the total entropy as a function of centrality/to the initial temperature profile?
7) What's the impact of resonance decays?

In [ ]:
using LaTeXStrings


LF=Fluidum.get_LF_Attributes(;file_path=pwd()*"/kernels/")
Key_Tuple_LF=(:proton,:kaon,:pion)    
dic = (; zip(Key_Tuple_LF, LF)...) 

ptrange=[i for i in range(0.1,3,100)]
spectra=Fluidum.spectra_lf(fo,dic.pion,params,ptrange,decays=false)
thermal=[spectra[i][1].*ptrange[i] for i in eachindex(ptrange)]
spectra=Fluidum.spectra_lf(fo,dic.pion,params,ptrange,decays=true)
total=[spectra[i][1].*ptrange[i] for i in eachindex(ptrange)]
plot(ptrange,thermal,yscale=:log10,label=string(dic.pion.name)*", thermal",xlabel=L"p_T \mathrm{[GeV]}",ylabel=L"dN/2\pi dp_T \mathrm{[GeV^{-1}]}",lw=2)
plot!(ptrange,total,yscale=:log10,label=string(dic.pion.name)*", total",xlabel=L"p_T \mathrm{[GeV]}",ylabel=L"dN/2\pi dp_T \mathrm{[GeV^{-1}]}",lw=2)

#=
spectra=Fluidum.spectra_lf(fo2,dic.pion,params,ptrange,decays=false)
thermal=[spectra[i][1].*ptrange[i] for i in eachindex(ptrange)]
spectra=Fluidum.spectra_lf(fo2,dic.pion,params,ptrange,decays=true)
total=[spectra[i][1].*ptrange[i] for i in eachindex(ptrange)]
plot!(ptrange,thermal,yscale=:log10,label=string(dic.pion.name)*", thermal",xlabel=L"p_T \mathrm{[GeV]}",ylabel=L"dN/2\pi dp_T \mathrm{[GeV^{-1}]}",lw=2)
plot!(ptrange,total,yscale=:log10,label=string(dic.pion.name)*", total",xlabel=L"p_T \mathrm{[GeV]}",ylabel=L"dN/2\pi dp_T \mathrm{[GeV^{-1}]}",lw=2)
=#

## Comparison with experimental data

7. Download the experimental data for pion transverse momentum distributions from the ALICE collaboration from HEPdata.

5.02 TeV PbPb: https://www.hepdata.net/record/ins1759506?version=1&table=Table%201
2.76 TeV: https://www.hepdata.net/record/ins1377750

Download them in the CSV format.

8. Plot the data together with the model calculations. What can you see? What is the effect of resonance decays?

In [ ]:
#import Pkg; Pkg.add("CSV")
#import Pkg; Pkg.add("DataFrames")
using CSV, DataFrames
df = CSV.File("HEPData-ins1759506-v1-Table_1.csv"; comment = "#") |> DataFrame
rename!(df,[:pt,:ptm,:ptp,:dNdpt,:statplus,:statmin,:systplus,:systmin,:systunplus,:systunmin])
pt = parse.(Float64, first(df.pt,36))
dNdpt = parse.(Float64, first(df.dNdpt,36))
yerrp=parse.(Float64, first(df.statplus,36))
yerrm=parse.(Float64, first(df.statmin,36))
xerrm=pt.-parse.(Float64, first(df.ptm,36))
plot(ptrange,4*pi*thermal,yscale=:log10,label=string(dic.pion.name)*", thermal",xlabel=L"p_T \mathrm{[GeV]}",ylabel=L"dN/2\pi dp_T \mathrm{[GeV^{-1}]}",lw=2)
plot!(ptrange,4*pi*total,yscale=:log10,label=string(dic.pion.name)*", total",xlabel=L"p_T \mathrm{[GeV]}",ylabel=L"dN/2\pi dp_T \mathrm{[GeV^{-1}]}",lw=2)
scatter!(pt,dNdpt,xerr = xerrm, yerror=(yerrm,yerrp),yscale=:log10,markerstrokewidth = 0.5,markersize = 2,label="data")

## Prequel to Bayesian Inference

To extract QGP properties by a comparison between model and data, it is important to undersand what observables are sensitive to which fluid properties. By looking at the particle spectra, to which parameter would you say it is sensitive out of the ones used today?